In [35]:
import pandas as pd
import requests
import wget
import time
import os
from bs4 import BeautifulSoup

add_col = ["代號","會計項目"]

co_id_dict = {	
    "晶華" :2707
    # "遠雄來" :2712, 
    # "夏都" :2722,
    # "美食-KY" :2723,
    # "王品" :2727,
    # "雄獅" :2731,
    # "寒舍" :2739,
    # "雲品" :2748,
    # "八方雲集" :2753, 
    # "鳳凰" :5706,
    # "柏文" :8462,
    # "新天地" :8940,
    # "好樂迪" :9943
}


def file_download(company_id,year,season):
    url = f'https://mops.twse.com.tw/server-java/t164sb01?step=3&SYEAR={year}&file_name=tifrs-fr1-m1-ci-cr-{company_id}-{year}Q{season}.html'
    rq = requests.get(url)
    rq.encoding = "big5"
    f = open(f'../temp/{company_id}_{year}_{season}', 'w')
    f.write(rq.text)
    f.close()
    time.sleep(4)

def clean_df(company_id,year,season):
    raw_xbrl = pd.read_html(f"https://mops.twse.com.tw/server-java/t164sb01?step=3&SYEAR={year}&file_name=tifrs-fr1-m1-ci-cr-{company_id}-{year}Q{season}.html",flavor="bs4")
    time.sleep(4)
    for df_num in range(0,4):
        if df_num != 3:
            mutiple_col=raw_xbrl[df_num].columns
            raw_xbrl[df_num].columns = [col_name[1]for col_name in mutiple_col]
            if df_num == 1:
                raw_xbrl[df_num].to_csv(f"../Financial_statements/{co_id}/Balance_Sheet/{company_id}_{year}_{season}.csv")
            elif df_num == 2:
                raw_xbrl[df_num].to_csv(f"../Financial_statements/{co_id}/Profit_and_Loss_Account/{company_id}_{year}_{season}.csv")
            else:
                raw_xbrl[df_num].to_csv(f"../Financial_statements/{co_id}/Cash_Flow_Statement/{company_id}_{year}_{season}.csv")
        else:
            mutiple_col=raw_xbrl[df_num].columns
            new_col = [col_name[2]for col_name in mutiple_col]
            new_col[0:2]=add_col
            raw_xbrl[df_num].columns = new_col
            raw_xbrl[df_num].to_csv(f"../Financial_statements/{co_id}/Statement of changes in equity/{company_id}_{year}_{season}.csv")

for co_id in co_id_dict.values():
    try:
        os.mkdir(f"../Financial_statements/{co_id}")
        os.mkdir(f"../Financial_statements/{co_id}/Balance_Sheet")
        os.mkdir(f"../Financial_statements/{co_id}/Profit_and_Loss_Account")
        os.mkdir(f"../Financial_statements/{co_id}/Cash_Flow_Statement")
        os.mkdir(f"../Financial_statements/{co_id}/Statement of changes in equity")
    except:
        pass
    for year in range(2019,2020):
        for season in range(4,5):
            try:
                clean_df(co_id,year,season)
            except:
                pass

